In [1]:
import os

In [2]:
os.chdir('../')

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class CallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [8]:
@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path # to store the trained model
    base_model_path: Path # load the base model
    
    train_path: Path
    test_path: Path
    valid_path: Path

    img_size: list
    epochs: int
    batch_size: int
    learning_rate: float


In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [17]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_callback_config(self) -> CallbacksConfig:
        config = self.config.callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        callback_config = CallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return callback_config

        
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        params = self.params
        
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
        create_directories([
            Path(training.root_dir)
        ])


        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            base_model_path=Path(training.base_model_path),
            train_path=Path(training.train_path),
            test_path=Path(training.test_path),
            valid_path=Path(training.valid_path),
            img_size=params.IMAGE_SIZE,
            epochs = params.epochs,
            batch_size=params.batch_size,
            learning_rate=params.LEARNING_RATE
        )

        return training_config

In [18]:
config = ConfigurationManager()
training_config = config.get_training_config()

[2023-12-30 23:30:36,072: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-30 23:30:36,076: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-30 23:30:36,076: INFO: common: created directory at: artifacts]
[2023-12-30 23:30:36,077: INFO: common: created directory at: artifacts/training]


In [19]:
training_config

TrainingConfig(root_dir=PosixPath('artifacts/training'), trained_model_path=PosixPath('artifacts/training/trained_model.keras'), base_model_path=PosixPath('artifacts/base_model/base_model_updated.keras'), train_path=PosixPath('artifacts/data_ingestion/smoke_data/Training/Training'), test_path=PosixPath('artifacts/data_ingestion/smoke_data/Testing/Testing'), valid_path=PosixPath('artifacts/data_ingestion/smoke_data/Validation/Validation'), img_size=BoxList([224, 224]), epochs=5, batch_size=32, learning_rate=0.001)

In [20]:
import os
import tensorflow as tf
import keras
import time
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd

In [21]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),
            save_best_only=True,
            monitor='val_accuracy',
            mode='max'
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [24]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
        self.model = None
        self.history = None
        self.train_gen = None
        self.valid_gen = None
        self.test_gen = None
        self.train_data_aug_20 = None
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.base_model_path
        )
        return self.model

    @staticmethod
    def extract_from_path(path)-> []:
        full_path = []
        for i in sorted(os.listdir(path)):
            full_path.append(os.path.join(path, i))
        return full_path

    @staticmethod
    def createDataFrame(image_paths):
        labels = [path.split('/')[-1].split('_')[0] for path in image_paths]
        df = pd.DataFrame({'path': image_paths, 'label': labels})
        df['label_id'] = df['label'].apply(lambda x: 1 if x == 'smoking' else 0)
        return df

    def get_train_test_valid_df(self):
        train_images = self.extract_from_path(self.config.train_path)
        test_images = self.extract_from_path(self.config.test_path)
        valid_images = self.extract_from_path(self.config.valid_path)
        train_images_df = self.createDataFrame(train_images)
        test_images_df = self.createDataFrame(test_images)
        valid_images_df = self.createDataFrame(valid_images)

        return train_images_df, test_images_df, valid_images_df

    
    def imgPreProcessing(self, image, label):
        img = tf.io.read_file(image)
        img = tf.io.decode_jpeg(img, channels = 3)
        img = tf.image.resize(img, size = (self.config.img_size))
        return img, label

        
    def loadDataset(self, df:pd.DataFrame):
        dataset = tf.data.Dataset.from_tensor_slices((df['path'], df['label_id']))
        return (dataset
                     .map(self.imgPreProcessing)
                    .shuffle(self.config.batch_size * 20)
                     .batch(self.config.batch_size))

    
    def get_data_generator(self, train_df, valid_df, test_df):
        train_gen = self.loadDataset(train_df)
        test_gen = self.loadDataset(test_df)
        valid_gen = self.loadDataset(valid_df)
        self.train_gen = train_gen
        self.test_gen = test_gen
        self.valid_gen = valid_gen
        
        return train_gen, test_gen, valid_gen
        

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
        
    @staticmethod
    def augmentation(image, label):
        SEED = 42
        img = tf.image.random_flip_left_right(image, seed = SEED)
        img = tf.image.random_brightness(img, 0.1,seed = SEED)
        img = tf.image.random_contrast(img, 0.2, 0.5,seed = SEED)
        img = tf.image.random_saturation(img, .5, 1,seed = SEED)
        img = tf.image.random_hue(img, 0.2,seed = SEED)
        return img, label

    
    def loadDatasetWithAugmentaion(self, df:pd.DataFrame):
        dataset = tf.data.Dataset.from_tensor_slices((df['path'], df['label_id']))
        return (dataset
                     .map(self.imgPreProcessing)
                     .map(self.augmentation)
                    .shuffle(self.config.batch_size * 20)
                     .batch(self.config.batch_size)
               )

    def get_augmentated_df(self, train_df):
                # Loading only 25 % for data augmentaion
        data_size = len(train_df)
        self.train_data_aug_20 = self.loadDatasetWithAugmentaion(train_df.sample(frac=1)[:int(0.25 * data_size)])
    
    def train(self, callback_list: list):

        self.model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(),optimizer = tf.keras.optimizers.Adam(), metrics = ["accuracy"])
        self.history = self.model.fit(
                                        self.train_data_aug_20,epochs = self.config.epochs, 
                                        steps_per_epoch = len(self.train_data_aug_20),
                                        validation_data = self.valid_gen, 
                                        validation_steps = len(self.valid_gen),
                                        callbacks = callback_list
                                      )
        # Fine tune the EfficientNetB0 layer
        print("Fine tune the EfficientNetB0 layer")
        base_eff_modelTwo_base = self.model.layers[1]
        base_eff_modelTwo_base.trainable = True
        # Freeze all layers except for the last 10
        for layer in base_eff_modelTwo_base.layers[:-10]:
            layer.trainable = False

        for no,layer in enumerate(base_eff_modelTwo_base.layers):
            print("Layer no : ",no,"Trainable : ",layer.trainable, "Layer Name : ", layer.name,)

        print("Total trainable parameters in the model ", len(self.model.trainable_variables))
            # Initila 2 + now unfreeing 10 = 12

        # recompiling the model
        self.model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(), optimizer = tf.keras.optimizers.Adam(learning_rate = self.config.learning_rate),metrics = ['accuracy'])
        fine_tune_epochs = self.config.epochs + 5
        self.history = self.model.fit(self.train_gen,epochs = fine_tune_epochs,
                                                          steps_per_epoch = len(self.train_gen),
                                                          initial_epoch=self.history.epoch[-1],
                                                          validation_data = self.valid_gen,
                                                          validation_steps = len(self.valid_gen),
                                                          callbacks = callback_list    
                                                  )    
        
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [25]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    model = training.get_base_model()
    train_df, valid_df, test_df = training.get_train_test_valid_df()
    train_gen, valid_gen, test_gen = training.get_data_generator(train_df, valid_df, test_df)

    training.get_augmentated_df(train_df)
    
    callbacks_config = config.get_callback_config()
    prepare_callbacks = PrepareCallback(config=callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2023-12-30 23:32:38,698: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-30 23:32:38,701: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-30 23:32:38,702: INFO: common: created directory at: artifacts]
[2023-12-30 23:32:38,703: INFO: common: created directory at: artifacts/training]
[2023-12-30 23:32:40,859: WARNING: optimizer: Skipping variable loading for optimizer 'Adam', because it has 5 variables whereas the saved optimizer has 1 variables. ]
[2023-12-30 23:32:43,999: INFO: common: created directory at: artifacts/callbacks/checkpoint_dir]
[2023-12-30 23:32:44,000: INFO: common: created directory at: artifacts/callbacks/tensorboard_log_dir]
Epoch 1/5
6/6 [==============================] - 14s 2s/step - loss: 0.8050 - accuracy: 0.4134 - val_loss: 0.6518 - val_accuracy: 0.5982
Epoch 2/5
6/6 [==============================] - 7s 1s/step - loss: 0.6389 - accuracy: 0.6257 - val_loss: 0.5616 - val_accuracy: 0.7455
Epoch 3/5
6/6 [=============